# LangChain: Memory

## Outline
* ConversationBufferMemory
* ConversationBufferWindowMemory
* ConversationTokenBufferMemory
* ConversationSummaryMemory

## ConversationBufferMemory

In [6]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


In [8]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [9]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


"Hello Andrew! It's nice to meet you. How can I assist you today?"

In [10]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'1+1 is equal to 2.'

In [6]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI: The answer to 1+1 is 2.
Human: What is my name?
AI:

> Finished chain.


'Your name is Andrew, as you mentioned earlier.'

In [7]:
print(memory.buffer)

Human: Hi, my name is Andrew
AI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI: The answer to 1+1 is 2.
Human: What is my name?
AI: Your name is Andrew, as you mentioned earlier.


In [11]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Andrew\nAI: Hello Andrew! It's nice to meet you. How can I assist you today?\nHuman: What is 1+1?\nAI: 1+1 is equal to 2."}

In [12]:
memory = ConversationBufferMemory()

In [13]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [14]:
print(memory.buffer)

Human: Hi
AI: What's up


In [15]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

In [17]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool Dude"})

In [18]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool\nHuman: Not much, just hanging\nAI: Cool Dude"}

## ConversationBufferWindowMemory

In [19]:
from langchain.memory import ConversationBufferWindowMemory

In [20]:
memory = ConversationBufferWindowMemory(k=1)               

In [21]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})


In [22]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [23]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

In [24]:
conversation.predict(input="Hi, my name is Dan")

"Hello Dan! It's nice to meet you. How can I assist you today?"

In [25]:
conversation.predict(input="What is 2*Pi?")

'2*Pi is approximately 6.28318530718.'

In [26]:
conversation.predict(input="What is my name?")

"I'm sorry, but I don't have access to personal information."

## ConversationTokenBufferMemory

In [23]:
#!pip install tiktoken

In [28]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0)

In [29]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [30]:
memory.load_memory_variables({})

{'history': 'AI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

## ConversationSummaryMemory

In [42]:
from langchain.memory import ConversationSummaryBufferMemory


In [37]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=5)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [43]:
memory.load_memory_variables({})

{'history': 'System: The human greets the AI and the AI asks what\'s up. The human replies that they are just hanging. The AI responds with a simple "Cool." The human then asks about the schedule for the day. The AI informs the human about a meeting at 8am with the product team, the need to prepare a PowerPoint presentation, and time to work on the LangChain project. At noon, there is a lunch meeting with a customer who is driving from over an hour away to discuss the latest in AI, and the AI advises the human to bring their laptop to show the latest LLM demo. The human asks what would be a good demo to show during the lunch meeting. The AI suggests showcasing the latest Language Model demo, which can impress and engage the customer by demonstrating the AI\'s language understanding and generation capabilities.'}

In [44]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [45]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human greets the AI and the AI asks what's up. The human replies that they are just hanging. The AI responds with a simple "Cool." The human then asks about the schedule for the day. The AI informs the human about a meeting at 8am with the product team, the need to prepare a PowerPoint presentation, and time to work on the LangChain project. At noon, there is a lunch meeting with a customer who is driving from over an hour away to discuss the latest in AI, and the AI advises the human to bring their laptop to show the latest LLM demo. The human asks what would be a good demo to show during the lunch meeting. The AI suggests showcasing the latest Language Mo

"A good demo to show during the lunch meeting would be the latest Language Model demo. This demo can impress and engage the customer by demonstrating the AI's language understanding and generation capabilities."

In [46]:
memory.load_memory_variables({})

{'history': 'System: The human greets the AI and the AI asks what\'s up. The human replies that they are just hanging. The AI responds with a simple "Cool." The human then asks about the schedule for the day. The AI informs the human about a meeting at 8am with the product team, the need to prepare a PowerPoint presentation, and time to work on the LangChain project. At noon, there is a lunch meeting with a customer who is driving from over an hour away to discuss the latest in AI, and the AI advises the human to bring their laptop to show the latest LLM demo. The human asks what would be a good demo to show during the lunch meeting. The AI suggests showcasing the latest Language Model demo, which can impress and engage the customer by demonstrating the AI\'s language understanding and generation capabilities. A good demo to show during the lunch meeting would be the latest Language Model demo. This demo can impress and engage the customer by demonstrating the AI\'s language understand